# 統計學習初論 (Spring, 2019) Homework 3
截止日期: 9AM, 2019/4/9
請將HTML檔上傳至Ceiba作業區。回答作業時建議使用 "三明治" 答題法。也就是說，先說明要做什麼，然後列出程式碼與結果，最後說明這些結果的意義。作業自己做。嚴禁抄襲。不接受紙本繳交，不接受遲交。請以英文或中文作答。

## 第一題 [Classification via Generative Models]
(50%) We are going to explore the problem of identifying smartphone position through probabilistic generative models. Motion sensors in smartphones provide valuable information for researchers to understand its owners. An interesting (and more challenging) task is to identify human activities through the data recorded by motion sensors. For example, we want to know whether the smartphone owner is walking, running, or biking. In this homework problem, we are going to tackle a simpler problem. We want to know the static position of the smartphone. There are six possible positions:

Phoneonback: The phone is laying on the back of the phone with the screen pointing up (away from the ground).
Phoneonfront: The phone is laying on the back of the phone with the screen pointing towards the ground
Phoneonbottom: The phone is standing on the bottom of the screen, meaning the bottom is pointed towards the ground
Phoneontop: The phone is standing on the top of the screen, meaning the top is pointed towards the ground
Phoneonleft: The phone is laying on the left side of the screen.
Phoneonright: The phone is laying on the right side of the screen.
The input data is the reading of the accelerometer (cf. https://en.wikipedia.org/wiki/Accelerometer) in the smartphone. We have a training dataset that contains about 28,500 data points for phones in each of the six positions. The following is some basic information of the training dataset.

### Model Training
To train probabilistic generative classifiers, we need to assume how the data are generated in one given position. Here we are going to adopt a simple assumption that the data are generated from multivariate Gaussian distributions. To train a model under this assumption, we need to compute the mean and covariance of features at each label in the training data.

For each of the six labels, you should record the following information:

mu: the mean vector.
cov: the covariance matrix.
prec: the inverse of covariance matrix.
detcov: logarithm of the determinant of the covariance matrix.
n: number of observations for this label.
prior: learned prior probability of this label.
You should store your learned parameters in a two-level dictionary structure. For example, you can access the precision of "Phoneonback" by amodel["Phoneonback"]["prec"], where amodel is the learned model.

### Prediction
To predict phone positions, compute the posterior of each position given the trained model, and select the position with the largest probability. That is, given the reading  g=(gx,gy,gz)T , the probability that the phone position is k can be written as  Prob(POS=k│g)∝Prob(g│POS=k)Prob(POS=k) .

The likelihood  Prob(g|POS=k)  is simply  N(g|μk,σk) , where  μk  and  σk  are learned from the training data.

Answer the following questions:

### Q1.1: Create your mypgc class.

- def fit(self, x_train, y_train): fit training data and calculate attributes of data (mean, covariance...).
- def predict(self, fit): predict class according to input testing data.
- def cond_pro(self, x, cls): calculate conditional probability of class cls when having input data x.

In [3]:
class mypgc() :
    
    def __init__(self):
        self.train_dict = dict()
    
    def std(self, data):
        ret = data.copy()
        ret =  (ret - np.mean(ret, axis = 0))/np.std(ret, axis = 0)
        return ret
    
    def fit(self, x_train, y_train):
        x_train_std = self.std(x_train)
        label, index = np.unique(y_train, return_inverse=True)
        group_x = dict()
        count = 0
        for i in index:
            if group_x.get(label[i]) != None:
                group_x[label[i]].append(x_train[count])
            else:
                group_x[label[i]] = [x_train[count]]
                self.train_dict[label[i]] = dict()
            count += 1
#         print('mu', 'cov', 'prec', 'detcov', 'n', 'prior')
        for key, value in group_x.items():
            self.train_dict[key]['mu'] = np.mean(value, axis = 0)
            self.train_dict[key]['cov'] = np.cov(np.array(value), rowvar = False)
            self.train_dict[key]['prec'] = np.linalg.inv(self.train_dict[key]['cov'])
            self.train_dict[key]['detcov'] = np.log(np.linalg.det(self.train_dict[key]['cov']))
            self.train_dict[key]['n'] = len(value)
            self.train_dict[key]['prior'] = self.train_dict[key]['n'] /count
#             print(key, self.train_dict[key]['mu'],self.train_dict[key]['cov'],self.train_dict[key]['prec'], self.train_dict[key]['detcov'],self.train_dict[key]['n'],self.train_dict[key]['prior'])

    def predict(self, x):
        max_likely = -10000
        ret_cls = ''
        for label in LABEL:
            prob = self.cond_pro(x, label)
            if prob > max_likely:
                max_likely = prob
                ret_cls = label
        return ret_cls
    
    def cond_pro(self, x, cls):
        u = self.train_dict[cls]['mu'] 
        u = u.reshape(len(u), 1)
        wk = np.dot(self.train_dict[cls]['prec'], u)
        wk0 = -0.5 * np.dot(np.dot(u.T, self.train_dict[cls]['prec']), u)[0]  + np.log(self.train_dict[cls]['prior'] )
        ak = np.dot(wk.T, x)[0] + wk0 - 0.5 * self.train_dict[cls]['detcov']
        numerator = ak[0]
        return numerator 

### Q1.2: Load data from "phone_train.picke" and train your model. List your learned model parameters in a pretty, human readable way so that the TA can easily check the correctness of your results.

In [4]:
import pandas as pd
import pickle
import numpy as np
import math


LABEL = ['Phoneonleft', 'Phoneonfront', 'Phoneonback', 'Phoneonbottom', 'Phoneontop', 'Phoneonright']

with open('phone_train.pickle', 'rb') as fh1:
    train_data = pickle.load(fh1)
y_train = train_data['label'].values
x_train = train_data.drop(['label'], axis=1).values
pgc1 = mypgc()
pgc1.fit(x_train, y_train)

|class|   mu (x, y, z)|     cov       |   prec  |  
|:---:|:-----------:|:-------------:|:---------:|
|Phoneonback| [ 0.20693328 -0.02615235  9.7963762 ] |  [ 0.00299006, -0.00195813,  0.00284766<br>[-0.00195813,  0.00229887, -0.0024254 <br> 0.00284766, -0.0024254,   0.0048876 ]| [ 943.63669582,  469.54782044, -316.78479596<br> 469.54782044, 1146.63145078,  295.42598486<br>[-316.78479596,  295.42598486,  535.76812508]  |  
|Phoneonbottom| [0.19076794 9.78584734 0.1465572 ] | [ 0.00273883, -0.0044379,   0.00151984<br>-0.0044379,   0.01026786, -0.00328148<br>0.00151984, -0.00328148,  0.00247968]|[1229.24902343,  503.42022589,  -87.22678994<br>503.42022589,  374.93550309,  187.61601574<br> -87.22678994,  187.61601574,  705.02302834] |  
|Phoneonfront | [ 0.11270401  0.14265129 -9.73579633] | [ 0.00173617, -0.00562249,  0.00071319<br>-0.00562249,  0.03069977, -0.00386164<br> 0.00071319, -0.00386164,  0.00185582] |   [1415.5956848,   258.4845593,    -6.15206605<br>258.4845593,    91.32078358,   90.68682325<br>  -6.15206605,   90.68682325,  729.91124637]| 
|Phoneonleft | [ 9.92639446,  0.09275717, -0.01933473]] |[ 0.00192839, -0.00619274,  0.00063551<br>-0.00619274,  0.03200844, -0.00317611<br> 0.00063551, -0.00317611,  0.00171996] |    [1369.95113577,  263.0134811,   -20.49603676<br> 263.0134811,    88.74589895,   66.6992188 <br> -20.49603676,   66.6992188 ,  712.14869845] |  
|Phoneonright |[-9.64916017e+00,  7.86873275e-02, -7.71762134e-03] | [ 0.00076675, -0.00067075,  0.00068673<br>-0.00067075,  0.00682422, -0.00627087<br> 0.00068673, -0.00627087,  0.00773266] |    [1432.0246564,    93.75083518,  -51.14931824<br>93.75083518,  581.24293978,  463.03773646<br>-51.14931824,  463.03773646,  509.36831176] |  
|Phoneontop | [ 8.13769555e-04 -9.69990841e+00 -1.00215800e-01]| [ 0.00184664,  0.00529325, -0.00601384<br> 0.00529325,  0.02519526, -0.02792123<br>-0.00601384, -0.02792123,  0.0331182 ] |    [1380.20106989, -186.01874711,   93.79832935<br>-186.01874711,  629.12383998,  496.62195539<br>93.79832935,  496.62195539,  465.9185559 ] |  

|class|   detcov|     n       |   prior  |  
|:---:|:-----------:|:-------------:|:------:|
|Phoneonback| -18.987516240382195 | 28566 | 0.17095459523510295 |  
|Phoneonbottom| -18.242307571926013|  27842   |  0.1666217825574367 |  
|Phoneonfront | -17.331692277173115 | 29079 |    0.1740246683064328 | 
|Phoneonleft |-27.60788503403908 | 29522 |     0.17667582302494958 |  
|Phoneonright | -18.48369965825518 | 25687  |    0.1537250818386925 |  
|Phoneontop | -17.04131855952812 | 26401 |   0.15799804903738546 | 


### Q1.3: Load the test data from "phone_test1.pickle" and apply your predict method. List the first 20 predictions and well as the correct labels. Compute the accuracy for your model. Accuracy is the number of correct predictions divided by total number of data points.

In [5]:
with open('phone_test1.pickle', 'rb') as fh2:
    test_data = pickle.load(fh2)
y_test = test_data['label'].values
x_test = test_data.drop(['label'], axis=1).values
# print('predicted outcome', '    ','truth value')
count = 0 
for i in range(len(x_test)):
    if pgc1.predict(x_test[i]) == y_test[i]:
        count += 1
print(count / len(x_test))
#     print(i, '  ', pgc1.predict(x_test[i]),'         ', y_test[i])
#     if i == 19:
#         break

1.0


    predicted outcome          truth value
    0     Phoneonfront         Phoneonfront
    1     Phoneonbottom        Phoneonbottom
    2     Phoneonbottom        Phoneonbottom
    3     Phoneonbottom        Phoneonbottom
    4     Phoneonfront         Phoneonfront
    5     Phoneonright         Phoneonright
    6     Phoneonfront         Phoneonfront
    7     Phoneontop           Phoneontop
    8     Phoneonfront         Phoneonfront
    9     Phoneonfront         Phoneonfront
    10    Phoneonfront         Phoneonfront
    11    Phoneonback          Phoneonback
    12    Phoneonleft          Phoneonleft
    13    Phoneonback          Phoneonback
    14    Phoneonbottom        Phoneonbottom
    15    Phoneonback          Phoneonback
    16    Phoneonright         Phoneonright
    17    Phoneonright         Phoneonright
    18    Phoneontop           Phoneontop
    19    Phoneonleft          Phoneonleft
    
The accuracy is 100%.

## 第二題 [Logistic Regression with L2 Regularization]
(100%) We discussed the Logistic regression and its variation, Bayesian logistic regression that adopts L2 regularization. The Logistic regression with L2 regularization minimize the following error function (cf. file 0b2 linear model for classification.pdf):

λ2wTw−∑ni=1[tnlnyn+(1−tnln(1−yn)], 
where  yn=11+exp(−wTxn)  and  tn∈{0,1}  is the label value,  xn  is the feature vector, and  w  is the regression coefficient vector.

We are going to consider an extension of this model to allow different level of regularization for different regression coefficients. Consider the constant term versus other features. The coefficient of the constant term is usually not regularized in logistic regression. It is because the constant term is associated with the prior class distribution (see the discussion in generative models), and regularizing this term will force the probability of the positive class given a zero feature vector to be 0.5. This may hurt the prediction ability since the true prior class probability may indicate other more reasonable values.

Another consideration is regarding the continuous-valued features and binary-valued features. We typically normalize continuous-valued features to have zero means and unit variances but keep binary-value features untouched. It makes sense to have a single regularization value for the continuous-valued features since all of them have been normalized. Similarly, if we do not have additional information, then all binary-valued features can have the same level of regularization. However, using the same regularization coefficient for the continuous-valued and binary-valued features may not be reasonable. That is, it is often beneficial to have a regularization coefficient for the continuous-valued features, and another regularization coefficient for the binary-valued features.

The above discussion suggests that a more sophisticated way to regularize a logistic regression is to have three regularization coefficients: 0 for the constant,  a1  for continuous-valued features, and  a2  for the binary-valued features. It is possible to further refine the regularization coefficients. However, hyper-parameter tuning associated with more regularization coefficients may be costly.

To achieve this goal, we are going to consider a variation of L2-regularized logistic regression that allow different level of regularization for each coefficient. In the following discussion, we are going to use  X  to denote the feature matrix in the training data. The i-th row in  X ,  xi , is the feature vector for the i-th training data. The last column of  X  is one unless the use does not want to include the constant term.

In this model, each regression coefficient may be associated with a different regularization coefficient. Bearing with the risk of ambigulity, we (again) use the scalar  λi  to denote the regularization coefficient for  wi . The vector  w=[w1,w2,...,wD]T  is the regression coefficient vector. Let  Λ  denote the diagonal matrix that have  λi  at  Λii . Our new error function becomes:

E(w)=12wTΛw−∑ni=1[tnlnyn+(1−tnln(1−yn)], 
where  yn=11+exp(−wTxn) .

This model allows  wi  to have regularization coefficient  λi . If the constant term is the last element in  w , then setting  λD  to  0  allow us to unregularize the constant term. We can simply set  λi  associated with continuous-valued features to one value, and elements associated with binary-value features to another value. This will achieve our goal of a more refined regularization structure.

Following the PRML text book and the class discussion, we are going to train the model using the Newton-Raphson optimization method. In order to do so, you need to derive the gradient and hession of  E(w) . Given the training dataset, we can optimize  w  via

w(new)=w(old)−H−1∇E 
In order to do so, we need to have an initial vector of  w  to kick start the iteration. One way to do this is to use the closed-form solution of ridge regression:  w=(XTX+bI)−1XTt , where  t  is the vector of training lables. Simply set  b  to the average of  λi . Another way is to change the original L2 regularization term in ridge regression to  12wTΛw  and derive the new closed form solution that mathces our model.

Create a Python class named mylogistic_l2 that perform model training and prediction.



In [37]:
# The sample usage should be like the following:
logic1 = mylogistic_l2(reg_vec = lambda_vec, max_iter = 1000, tol = 1e-5, add_intercept = True)
logic1.fit(X_train, Y_train)
ypred = logic1.predict(X_test)

The first line is to create an object with the specified regularization coefficient vector, lambda_vec, and set the maximum number of iteration to 1000. The "tol" parameter set the stopping condition for Newton-Raphson optimization. The iteration will stop if the improvement on the error function is less than  10−5 . The "add_intercept" option says that we need add a column of ones to the end of X_train before model training. The length of lambda_vec, as a result should match the number of columns after adding the "one column" when this option is turned on.

To simplify the discussion, we use 0.5 as the threshold for the positive case when making prediction.

#### Implementation Restrictions
You are allowed to use the "building block" libraries including numpy and scipy in your own mylogistic_l2 class. You will receive a zero score if you adopted an existing logistic regression classifier in your answer. The input features and lables for the train method should be numpy arrays. The input features and output lables for the predict method should be numpy arrays.

#### Dataset
We are going to use to "Adult" dataset on the UCI machine learning reposition https://archive.ics.uci.edu/ml/datasets/Adult. The goal is to predict the label values of the income column, which can be either '>50K' or '<=50K.' The dataset had splitted the training and test data, and we are going to respect this particular train-test split in model testing.

To use this dataset in our model testing, you need to go through the data cleaning process so that the label value will be 1 for '>50K' and 0 otherwise. You should remove all rows with missing values. That is, if a row contain any missing values, remove that row. All features with discrete-values (for example, native-country and workclass) should be converted to "1-of-K" encoding. Include a particular feature value only if this unique value appears more than 10 times in the training data.

### Q2.1 (20%) Download the Adult dataset. Clean up the dataset and create x_train, y_train, x_test, y_test for training feature, training value, test feature, test label. All of these variables should be numpy arrays. Provide summary statistics for your training and test datasets so that TA can verify the correctness of your procedure.

In [2]:
import pandas as pd
import pickle
import numpy as np
import math


COLUMN_NAMES = ['age','workclass','fnlwgt','education','education-num','marital-status','occupation','relationship','race','sex','capital-gain','capital-loss','hours-per-week','native-country', 'result']
CONTINUOUS = ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']
RESULT = ['result']
DESCRETE = list(set(COLUMN_NAMES) - set(CONTINUOUS) - set(RESULT))


### Training data processed result:

In [445]:
import pandas as pd
import pickle
import numpy as np
import math

train_data = pd.read_csv('adult_data.txt', sep = '\, ', names = COLUMN_NAMES, engine = 'python')
cleaned_train_data = data_cleaning(train_data, 'train')
binary_train_data = pd.DataFrame()
drop_fea = set()
for i in DESCRETE:
    dummy = pd.get_dummies(cleaned_train_data[i])
    binary_train_data = pd.concat([binary_train_data, dummy], axis = 1)
for col in binary_train_data.columns:
    if binary_train_data[col].sum() <= 10:
        drop_fea.add(col)
binary_train_data = binary_train_data.drop(drop_fea, axis = 1)
filter_train_data = cleaned_train_data[list(set(COLUMN_NAMES)- set(DESCRETE))].join(binary_train_data)

In [444]:
train_y = filter_train_data['result'].values
train_x = filter_train_data.drop('result',  axis=1).values

filter_train_data.drop('result',  axis=1).describe()


,capital-gain,fnlwgt,capital-loss,age,education-num,hours-per-week,Amer-Indian-Eskimo,Asian-Pac-Islander,Black,Other,...,Some-college,Federal-gov,Local-gov,Private,Self-emp-inc,Self-emp-not-inc,State-gov,Without-pay,Female,Male
count,30162.000000,3.016200e+04,30162.000000,30162.000000,30162.000000,30162.000000,30162.000000,30162.000000,30162.000000,30162.000000,...,30162.000000,30162.000000,30162.000000,30162.000000,30162.000000,30162.000000,30162.000000,30162.000000,30162.000000,30162.000000
mean,1092.007858,1.897938e+05,88.372489,38.437902,10.121312,40.931238,0.009482,0.029673,0.093396,0.007659,...,0.221404,0.031265,0.068530,0.738877,0.035608,0.082853,0.042404,0.000464,0.324315,0.675685
std,7406.346497,1.056530e+05,404.298370,13.134665,2.549995,11.979984,0.096915,0.169687,0.290991,0.087179,...,0.415199,0.174035,0.252657,0.439254,0.185313,0.275664,0.201513,0.021540,0.468126,0.468126
min,0.000000,1.376900e+04,0.000000,17.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,1.176272e+05,0.000000,28.000000,9.000000,40.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,1.784250e+05,0.000000,37.000000,10.000000,40.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,0.000000,2.376285e+05,0.000000,47.000000,13.000000,45.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
max,99999.000000,1.484705e+06,4356.000000,90.000000,16.000000,99.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


- Above picture shows train_x and train_y information.

### Testing data processed result:

In [79]:
test_data = pd.read_csv('adult_test.txt', sep = '\, ', names = COLUMN_NAMES, engine = 'python')
cleaned_test_data = data_cleaning(test_data, 'test')
binary_test_data = pd.DataFrame()
for i in DESCRETE:
    dummy = pd.get_dummies(cleaned_test_data[i])
    binary_test_data = pd.concat([binary_test_data, dummy], axis = 1)
for i in drop_fea:
    try:
        binary_test_data = binary_test_data.drop(i, axis = 1)
    except:
        pass
filter_test_data = cleaned_test_data[list(set(COLUMN_NAMES)- set(DESCRETE))].join(binary_test_data)

In [442]:
test_y = filter_test_data['result'].values
test_x = filter_test_data.drop('result',  axis=1).values
filter_test_data.drop('result',  axis=1).describe()

,capital-gain,fnlwgt,capital-loss,age,education-num,hours-per-week,Amer-Indian-Eskimo,Asian-Pac-Islander,Black,Other,...,Some-college,Federal-gov,Local-gov,Private,Self-emp-inc,Self-emp-not-inc,State-gov,Without-pay,Female,Male
count,15060.000000,1.506000e+04,15060.000000,15060.000000,15060.000000,15060.000000,15060.000000,15060.000000,15060.000000,15060.000000,...,15060.000000,15060.000000,15060.000000,15060.000000,15060.000000,15060.000000,15060.000000,15060.000000,15060.000000,15060.000000
mean,1120.301594,1.896164e+05,89.041899,38.768327,10.112749,40.951594,0.009894,0.027092,0.093692,0.008101,...,0.213878,0.030744,0.068592,0.731806,0.037981,0.086122,0.044290,0.000465,0.326228,0.673772
std,7703.181842,1.056150e+05,406.283245,13.380676,2.558727,12.062831,0.098977,0.162356,0.291409,0.089643,...,0.410055,0.172628,0.252768,0.443034,0.191158,0.280554,0.205744,0.021555,0.468848,0.468848
min,0.000000,1.349200e+04,0.000000,17.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,1.166550e+05,0.000000,28.000000,9.000000,40.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,1.779550e+05,0.000000,37.000000,10.000000,40.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,0.000000,2.385888e+05,0.000000,48.000000,13.000000,45.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
max,99999.000000,1.490400e+06,3770.000000,90.000000,16.000000,99.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


- Above picture shows test_x and text_y information.

In [6]:
def data_cleaning(data, types):
    ret = data.copy()
    ret = ret.replace('?', np.nan).dropna()
    if types == 'train':
        ret = ret.replace({'result':{'>50K':1, '<=50K': 0}})
    else:
        ret = ret.replace({'result':{'>50K.':1, '<=50K.': 0}})
    ret = ret.reset_index()
    return ret

### Q2.2 (20%) Derive the gradient and hession matrix for the new E(w).

- Gradient 
$$
(\sum_{n=1}^N (y_n - t_n) * x_n)  +  \lambda * w
$$

- Hession 
$$
(\sum_{n=1}^N y_n *( 1 - y_n) * x_n * x_n^T)  +  \lambda 
$$

$$
y_n = \sigma_( W^T *x)
$$

### Q2.3 (30%) Create your own mylogistic_l2 class. Show the learned w as well as test accuracy for the cases below. If w is too long for you, show selected w for continuous-valued, binary-valued, and the constant term.

In [580]:
import time

class mylogistic_l2():
    
    def __init__(self, reg_vec = None, max_iter = 1000, tol = 1e-5, add_intercept = True):
        self.reg_vec = reg_vec
        self.max_iter = max_iter
        self.tol = tol
        self.add_intercept = add_intercept
        self.train_x = None
        self.train_y = None
        self.w_coef = None
        
    def sigmoid(self, z):
#         z = np.array([*map(self.replace_extreme, z)])
        return 1.0 / (1.0 + np.exp(-z))

    def log_likelihood(self, x, y, w):  
        z = np.matmul(x, w).astype("float_")  
        sigmoid_probs = self.sigmoid(z)
        return -0.5 * np.matmul(np.matmul(w.T, self.reg_vec), w) + np.sum(np.dot(y, np.log(sigmoid_probs))+ np.dot((1 - y) , np.log(1 - sigmoid_probs)))
    
    def replace_extreme(self, val):
        eps = 0.000000001
        if val < eps:
            val = eps
        elif val > 1 - eps:
            val = 1 - eps
        return val
    
    def gradient(self, x, y, w,):    
        z = np.matmul(x, w).astype("float_")  
        sigmoid_probs = self.sigmoid(z)
        y = y.reshape(len(y), 1)
        # compute gradient
#         print(np.shape(np.matmul(np.transpose(x), sigmoid_probs - y)), np.shape(np.matmul(self.reg_vec, w)))
#         print(np.shape(np.matmul(np.transpose(x), sigmoid_probs - y) + np.matmul(self.reg_vec, w)))
        err_gra = np.matmul(np.transpose(x), sigmoid_probs - y)
        return err_gra + np.matmul(self.reg_vec, w)
    
    def hessian(self, x, w):
        z = np.matmul(x, w).astype("float_")  
        sigmoid_probs = self.sigmoid(z) 
        one_sub_sigmoid_probs = 1 - sigmoid_probs
        R =  (sigmoid_probs * one_sub_sigmoid_probs).flatten()
#         R_nn = np.diag(R)
        res = np.multiply(x.T, R)
        hes = np.matmul(res, x)
#         print(time.time())
#         hes = np.linalg.multi_dot( [ x.T, R_nn, x])
#         print(time.time())
        return (hes + self.reg_vec)
        
    def newtons_method(self, x, y): 
        # Initialize log_likelihood & parameters
        b = np.diag(self.reg_vec).mean()
        b = np.diag(b * np.array([1 for i in range(len(self.reg_vec))]))
        
        # The intercept term 
        w = np.linalg.multi_dot( [np.linalg.inv( np.matmul(x.T, x) + b ), x.T , y])
        w = w.reshape((len(w), 1))
        delta_l = np.Infinity                                          
        # Convergence Conditions                                                        
        tol = self.tol                                                               
        max_iter = self.max_iter                                                           
        i = 0
        l = self.log_likelihood(x, y, w)
         # w -> M x 1 matrix

        while abs(delta_l) > tol and i < max_iter :                                       
            i += 1                           
#             print('iteration : ', i)
            g = self.gradient(x, y, w)
            hess = self.hessian(x, w)
#             print(hess)
            H_inv = np.linalg.inv(hess)
            g_hess = np.matmul(H_inv, g)
            # Perform our update step
            w = w - g_hess                                                                
            # Update the log-likelihood at each iteration                                     
            l_new = self.log_likelihood(x, y, w)  
            delta_l = l - l_new 
            l = l_new
        return w                 

    def fit(self, x, y):
        self.train_x = x
        self.train_y = y
        if self.add_intercept:
            intercept = np.zeros((len(self.train_x), 1)) + 1
            self.train_x = np.array(np.append(self.train_x, intercept, axis=1))
        w_coef = self.newtons_method(self.train_x, self.train_y)
#         print('W coefficient is :\n',w_coef)
        self.w_coef = w_coef
        
    def predict(self, x):
        if self.sigmoid(np.matmul(x, self.w_coef))> 0.5 :
            return 1
        else:
            return 0

### Case 1: lambda = 1 for all coefficients

In [560]:
lambda_vec = np.identity(train_x.shape[1]+1)
print(lambda_vec)
# print(lambda_vec)
logic1 = mylogistic_l2(lambda_vec, 1000, 1e-5, True)
logic1.fit(train_x, train_y)

[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]
iteration :  1
iteration :  2
iteration :  3
iteration :  4
iteration :  5
iteration :  6
iteration :  7
W coefficient is :
 [[ 3.16590245e-04]
 [ 7.26281835e-07]
 [ 6.38679434e-04]
 [ 2.48591297e-02]
 [ 1.85806163e-01]
 [ 2.90215841e-02]
 [-1.02439618e+00]
 [-2.92668732e-01]
 [-6.35195701e-01]
 [-8.96890177e-01]
 [-4.84594618e-01]
 [ 9.03420736e-01]
 [ 4.04668560e-01]
 [-5.46920552e-01]
 [-1.31374667e+00]
 [ 4.27865170e-01]
 [-9.47705967e-01]
 [-1.07395557e-01]
 [-4.09492097e-01]
 [ 3.77034140e-01]
 [ 5.45751678e-01]
 [ 5.23589384e-01]
 [-6.70323916e-01]
 [-1.09516216e-01]
 [ 4.53277949e-02]
 [-1.63971889e-01]
 [-4.71983494e-02]
 [-4.02281380e-03]
 [-3.44708811e-01]
 [ 1.03362979e-01]
 [ 4.50497096e-01]
 [ 8.22875669e-01]
 [ 9.41389495e-02]
 [ 2.92357805e-01]
 [-3.50891475e-01]
 [-4.39859204e-01]
 [-4.07586321e-01]
 [-6.90804284e-0

In [540]:
count = 0
if logic1.add_intercept:
    intercept = np.array([[1] for i in range(len(test_x))])
    test_x_inter = np.append(test_x, intercept, axis=1)
for i in range(len(test_x)):
    res = logic1.predict(test_x_inter[i])
    if res == test_y[i]:
        count += 1
print('The accuracy is :',count/len(test_x))

The accuracy is : 0.8480743691899071


In [535]:
x1 = np.arange(9.0).reshape((3, 3))
x2 = np.arange(3.0)
print(x1,x2)
np.multiply(x1, x2)

[[0. 1. 2.]
 [3. 4. 5.]
 [6. 7. 8.]] [0. 1. 2.]


array([[ 0.,  1.,  4.],
       [ 0.,  4., 10.],
       [ 0.,  7., 16.]])

### Case 2: lambda = 1 for all but the intercept, no regularization for incercept term.

In [557]:
lambda_vec = np.identity(train_x.shape[1]+1)
lambda_vec [-1, -1] = 0
logic2 = mylogistic_l2(lambda_vec, 1000, 1e-5, True)
logic2.fit(train_x, train_y)

b
[[0.99029126 0.         0.         ... 0.         0.         0.        ]
 [0.         0.99029126 0.         ... 0.         0.         0.        ]
 [0.         0.         0.99029126 ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.99029126 0.         0.        ]
 [0.         0.         0.         ... 0.         0.99029126 0.        ]
 [0.         0.         0.         ... 0.         0.         0.99029126]]
iteration :  1
iteration :  2
iteration :  3
iteration :  4
iteration :  5
iteration :  6
iteration :  7
W coefficient is :
 [[ 3.17024478e-04]
 [ 7.50706804e-07]
 [ 6.39652126e-04]
 [ 2.54336822e-02]
 [ 2.95324923e-01]
 [ 2.94914512e-02]
 [-3.72048376e-01]
 [ 3.94360616e-01]
 [ 4.30693947e-02]
 [-2.61235809e-01]
 [ 1.95854174e-01]
 [ 1.00965472e+00]
 [ 5.02337197e-01]
 [-4.58396920e-01]
 [-1.24053337e+00]
 [ 5.27833699e-01]
 [-8.67620758e-01]
 [-2.75702542e-02]
 [-3.15345727e-01]
 [ 4.72832596e-01]
 [ 6.28755385e-01]
 [ 6.23911101e-01]
 [-5.884331

In [542]:
count = 0
if logic2.add_intercept:
    intercept = np.array([[1] for i in range(len(test_x))])
    test_x_inter = np.append(test_x, intercept, axis=1)
for i in range(len(test_x_inter)):
    res = logic2.predict(test_x_inter[i])
    if res == test_y[i]:
        count += 1
print('The accuracy is :',count/len(test_x))

The accuracy is : 0.847808764940239


### Case 3: lambda = 1 for numerical-valued features, lambda = 0.5 for binary-valued features, no regularization for incercept term.

In [568]:
lambda_vec = np.identity(train_x.shape[1]+1)
for i in range(len(lambda_vec)):
    if i >= 5:
        lambda_vec[i][i] = 0.5
lambda_vec [-1, -1] = 0
logic3 = mylogistic_l2(lambda_vec, 1000, 1e-5, True)
logic3.fit(train_x, train_y)

[[1.  0.  0.  ... 0.  0.  0. ]
 [0.  1.  0.  ... 0.  0.  0. ]
 [0.  0.  1.  ... 0.  0.  0. ]
 ...
 [0.  0.  0.  ... 0.5 0.  0. ]
 [0.  0.  0.  ... 0.  0.5 0. ]
 [0.  0.  0.  ... 0.  0.  0. ]]
iteration :  1
iteration :  2
iteration :  3
iteration :  4
iteration :  5
iteration :  6
iteration :  7
W coefficient is :
 [[ 3.17319918e-04]
 [ 7.51944447e-07]
 [ 6.40115467e-04]
 [ 2.54757380e-02]
 [ 3.19092334e-01]
 [ 2.95136416e-02]
 [-3.83664223e-01]
 [ 4.13058389e-01]
 [ 4.13741722e-02]
 [-2.63868110e-01]
 [ 1.93099772e-01]
 [ 1.18913504e+00]
 [ 5.50823621e-01]
 [-4.76703416e-01]
 [-1.45906507e+00]
 [ 5.82266388e-01]
 [-1.06198169e+00]
 [-9.41595511e-03]
 [-3.18451269e-01]
 [ 5.24213619e-01]
 [ 7.29262486e-01]
 [ 6.74417704e-01]
 [-6.38234075e-01]
 [-9.70370974e-03]
 [ 1.74243036e-01]
 [-2.36216830e-01]
 [ 3.80747425e-02]
 [ 1.00364860e-01]
 [-2.47188656e-01]
 [ 2.38206872e-01]
 [ 6.41987645e-01]
 [ 1.00609325e+00]
 [ 2.33159600e-01]
 [ 4.22777999e-01]
 [-3.53092034e-01]
 [-2.90744304e-01]

In [569]:
count = 0
if logic3.add_intercept:
    intercept = np.array([[1] for i in range(len(test_x))])
    test_x_inter = np.append(test_x, intercept, axis=1)
for i in range(len(test_x_inter)):
    res = logic3.predict(test_x_inter[i])
    if res == test_y[i]:
        count += 1
print('The accuracy is :', count/len(test_x))

The accuracy is : 0.847675962815405


### Q2.4 (10%) Further split the training data into subtraining (90%) and tuning (10%) to search for the best hyper-parameters. Set the regularization coefficient for the constant term to zero. Allow different regularizations for continuous-valued and binary-valued features. Let  a1  and  a2  denote the regularization coefficients for continuous-valued and binary-valued features. Search the best  a1  and  a2  and report the test accuracy using the best hyper-parameters. You should follow the following procedure to search for the best hyper-parameters.

    Choose a set of grids among a reasonable range. For example, 10 grids in [0.01, 100].
    Conduct grid search with the constraint that  a1=a2 . Record the best value  a∗1  and  a∗2 .
    Fix  a1=a∗1 , and search  a2  for the best value, call the result the new  a∗2 .
    Fix  a2=a∗2 , and search  a1  for the best value.
    Report the selected  a1  and  a2 .
    Train a model using the selected hyper-parameters, and report the test accuracy.

In [602]:
def get_lambda_vector(continuous, binary, intercept = 0):
    lambda_vec = list()
    for i in range(train_x.shape[1]+1):
        if i < 6:
            lambda_vec.append (continuous)
        elif i < train_x.shape[1]:
            lambda_vec.append(binary)
        else:
            lambda_vec.append(0)
        i += 1
    return  np.diag(lambda_vec)

def get_accuracy(logis, x, obs):
    correct = 0
    for i in range(len(obs)):
        if logis.predict(x[i]) == obs[i]:
            correct += 1
    return(correct/len(obs))

def find_best_a(subtrain_x, tuning_x, subtrain_y, tuning_y, a_set, a1 = None, a2 = None, ):
    best_accu = 0
    best_a = 0
    flag_1 = 0
    flag_2 = 0
    if a1 == None:
        flag_1 =1
    if a2 == None:
        flag_2 =1 
    for a in a_set:
        if flag_1 :
            a1 = a
        if flag_2:
            a2 = a
        lambda_vec = get_lambda_vector(a1, a2)
        mylogistic = mylogistic_l2(reg_vec = lambda_vec, max_iter = 1000, tol = 0.00001, add_intercept = True)
        if mylogistic.add_intercept:
            intercept = np.array([[1] for i in range(len(tuning_x))])
            tuning_x_inter = np.append(tuning_x, intercept, axis=1)
        mylogistic.fit(subtrain_x, subtrain_y)
        accuracy = get_accuracy(mylogistic, tuning_x_inter, tuning_y)
        if accuracy > best_accu:
            best_accu = accuracy
            best_a = a
        print(a, accuracy)
    return  best_a, best_accu

In [593]:
from sklearn.model_selection import train_test_split
import time 
subtrain_x, tuning_x, subtrain_y, tuning_y = train_test_split(train_x, train_y, test_size = 0.1)
a_set = np.logspace(-2, 2, 10) 
# print('a value set:\n', a_set)
a1, accu = find_best_a(subtrain_x, tuning_x, subtrain_y, tuning_y, a_set)
print(f'best a1=a2 is {a1}', f'accuracy is {accu}')
res_1 = a1

0.01 0.8508452104739808
0.027825594022071243 0.851508120649652
0.0774263682681127 0.8518395757374876
0.21544346900318834 0.8518395757374876
0.5994842503189409 0.8525024859131588
1.6681005372000592 0.8528339410009944
4.6415888336127775 0.8528339410009944
12.915496650148826 0.8511766655618164
35.93813663804626 0.8505137553861452
100.0 0.8491879350348028
best a1=a2 is 1.6681005372000592 accuracy is 0.8528339410009944


In [594]:
a2, accu = find_best_a(subtrain_x, tuning_x, subtrain_y, tuning_y, a_set, a1)
print(f'best when a1 fixed at {res_1}, a2 is {a2}', f'accuracy is {accu}')
res_2 = a2

0.01 0.8508452104739808
0.027825594022071243 0.851508120649652
0.0774263682681127 0.8518395757374876
0.21544346900318834 0.8518395757374876
0.5994842503189409 0.8525024859131588
1.6681005372000592 0.8528339410009944
4.6415888336127775 0.8528339410009944
12.915496650148826 0.8511766655618164
35.93813663804626 0.8505137553861452
100.0 0.8491879350348028
best when a1 fixed at 1.6681005372000592, a2 is 1.6681005372000592 accuracy is 0.8528339410009944


In [603]:
a1, accu = find_best_a(subtrain_x, tuning_x, subtrain_y, tuning_y, a_set, None, a2)
print(f'best when a2 fixed at {res_2}, a1 is {a1}', f'accuracy is {accu}')

0.01 0.8528339410009944
0.027825594022071243 0.8528339410009944
0.0774263682681127 0.8528339410009944
0.21544346900318834 0.8528339410009944
0.5994842503189409 0.8528339410009944
1.6681005372000592 0.8528339410009944
4.6415888336127775 0.8528339410009944
12.915496650148826 0.8528339410009944
35.93813663804626 0.8528339410009944
100.0 0.8528339410009944
best when a2 fixed at 1.6681005372000592, a1 is 0.01 accuracy is 0.8528339410009944


### Q2.5 (20%) Use sklearn.linear_model.LogisticRegression to train and test the model (including hyper-parameter tuning). Compare the estimated parameters and test accuracy with those from your own models.

In [44]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

grid={"C":np.logspace(-3,3,7), "penalty":["l2"], 'solver':['newton-cg']}
clf = LogisticRegression()
clf_cv = GridSearchCV(clf, grid, cv=10)
clf_cv.fit(np.array(train_x), np.array(train_y))


/miniconda3/lib/python3.7/site-packages/scipy/optimize/linesearch.py:462: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/miniconda3/lib/python3.7/site-packages/scipy/optimize/linesearch.py:313: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/miniconda3/lib/python3.7/site-packages/sklearn/utils/optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)
/miniconda3/lib/python3.7/site-packages/scipy/optimize/linesearch.py:462: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/miniconda3/lib/python3.7/site-packages/scipy/optimize/linesearch.py:313: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did

/miniconda3/lib/python3.7/site-packages/sklearn/utils/optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)
/miniconda3/lib/python3.7/site-packages/sklearn/utils/optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)
/miniconda3/lib/python3.7/site-packages/scipy/optimize/linesearch.py:462: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/miniconda3/lib/python3.7/site-packages/scipy/optimize/linesearch.py:313: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/miniconda3/lib/python3.7/site-packages/sklearn/utils/optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iteration

/miniconda3/lib/python3.7/site-packages/sklearn/utils/optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)
/miniconda3/lib/python3.7/site-packages/sklearn/utils/optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)
/miniconda3/lib/python3.7/site-packages/scipy/optimize/linesearch.py:462: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/miniconda3/lib/python3.7/site-packages/scipy/optimize/linesearch.py:313: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/miniconda3/lib/python3.7/site-packages/sklearn/utils/optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iteration

/miniconda3/lib/python3.7/site-packages/sklearn/utils/optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)
/miniconda3/lib/python3.7/site-packages/scipy/optimize/linesearch.py:462: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/miniconda3/lib/python3.7/site-packages/scipy/optimize/linesearch.py:313: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/miniconda3/lib/python3.7/site-packages/sklearn/utils/optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)
/miniconda3/lib/python3.7/site-packages/sklearn/utils/optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iteration

GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]), 'penalty': ['l2'], 'solver': ['newton-cg']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [45]:
print("best parameters: ",clf_cv.best_params_)
print("accuracy :",clf_cv.best_score_)

best parameters:  {'C': 100.0, 'penalty': 'l2', 'solver': 'newton-cg'}
accuracy : 0.8488495457860885


In [70]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(C= 100.0, penalty= 'l2', solver = 'newton-cg').fit(train_x, train_y)
res = clf.predict(test_x)
count = 0
for i in range(len(test_x)):
    if res[i] == test_y[i]:
        count += 1
print('accuracy :',count/len(test_x))

/miniconda3/lib/python3.7/site-packages/scipy/optimize/linesearch.py:462: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/miniconda3/lib/python3.7/site-packages/scipy/optimize/linesearch.py:313: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)


accuracy : 0.8479415670650731


/miniconda3/lib/python3.7/site-packages/sklearn/utils/optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)


By using sklearn package, I get highest accuracy with 0.8479415670650731